# Notebook

## Hal yang Dilakukan
Di notebook ini, saya melakukan serangkaian kegiatan untuk menganalisis dan mengolah data yang ada. Mulai dari mengecek keberadaan data, membersihkan data yang nilainya tidak jelas atau tidak ada, sampai memvisualisasikan data sementara sebelum membuat dashboard interaktif di Tableau.

## Hal yang Dihasilkan
Melalui notebook ini, saya akan menghasilkan data yang sudah bersih dan tidak acak lagi (**raw data**) guna membuat dashboard menjadi lebih baik dan memanjakan mata awwuuwwooo.

In [ ]:
# Data Collection

import pandas as jokowi

path = 'dataset/online-retail.csv'

print(f"Baca file: {path} ...")

df = jokowi.read_csv(path, encoding='ISO-8859-1')

print("berhasil")
print(f"Data: {df.shape[0]} baris, {df.shape[1]} kolom")
df.head()

Baca file: dataset/online-retail.csv ...
berhasil
Data: 541909 baris, 8 kolom


,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,12/1/2010 8:26,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,12/1/2010 8:26,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,12/1/2010 8:26,3.39,17850.0,United Kingdom


### Analisis Hasil

Dalam proyek ini, saya menggunakan dataset transaksi ritel daring berskala besar yang terdiri dari 541.909 baris data untuk menganalisis pola perilaku konsumen secara mendalam. Dataset ini memuat delapan variabel kunci, termasuk InvoiceDate yang saya manfaatkan untuk mengukur aspek Recency, CustomerID untuk mengidentifikasi Frequency belanja pelanggan unik, serta kombinasi Quantity dan UnitPrice untuk menghitung nilai Monetary. Struktur data yang komprehensif ini menjadi landasan utama bagi saya dalam melakukan segmentasi pelanggan (RFM Analysis) dan menggali wawasan bisnis dari aktivitas penjualan yang didominasi oleh pasar Inggris (United Kingdom) namun tetap mencakup transaksi internasional.

In [ ]:
# Cleaning data

import pandas as jokowi

print("Cleaning...")

df_clean = df.copy()

df_clean = df_clean.dropna(subset=['CustomerID'])

df_clean = df_clean[(df_clean['Quantity'] > 0) & (df_clean['UnitPrice'] > 0)]

df_clean['CustomerID'] = df_clean['CustomerID'].astype(int)

df_clean['InvoiceDate'] = jokowi.to_datetime(df_clean['InvoiceDate'])

df_clean['TotalPrice'] = df_clean['Quantity'] * df_clean['UnitPrice']

print("cleaning selesai")
print(f"data tersisa: {df_clean.shape[0]} baris")
df_clean.head()

Cleaning...
cleaning selesai
data tersisa: 397884 baris


,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,TotalPrice
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850,United Kingdom,15.30
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850,United Kingdom,22.00
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34


### Analisis

Dalam tahap persiapan data ini, saya melakukan serangkaian proses pembersihan untuk memastikan kualitas analisis RFM yang akurat dan relevan. Fokus utama saya adalah mengeliminasi transaksi yang tidak memiliki informasi CustomerID, karena analisis segmentasi pelanggan mustahil dilakukan tanpa mengetahui identitas unik dari setiap pembeli. Selain itu, saya juga membersihkan data dari gangguan teknis (noise) seperti transaksi dengan jumlah barang negatif yang mengindikasikan retur, serta harga yang tidak valid, agar perhitungan nilai moneter mencerminkan pendapatan riil perusahaan. Saya juga telah melakukan penyesuaian tipe data, khususnya pada format tanggal dan kode pelanggan, untuk memfasilitasi operasi matematika di tahap selanjutnya. Meskipun proses filtrasi ini menyusutkan volume data menjadi 397.884 baris, himpunan data yang tersisa kini memiliki integritas tinggi dan siap digunakan untuk memetakan perilaku pelanggan secara presisi.

In [5]:
tanggal_awal = df_clean['InvoiceDate'].min()
tanggal_akhir = df_clean['InvoiceDate'].max()

print(f"Data paling awal: {tanggal_awal}")
print(f"Data terakhir: {tanggal_akhir}")

durasi = tanggal_akhir - tanggal_awal
print(f"\nPeriode: {durasi.days} hari")

Data paling awal: 2010-12-01 08:26:00
Data terakhir: 2011-12-09 12:50:00

Periode: 373 hari


### Verifikasi
Melalui pemeriksaan rentang waktu data, saya memverifikasi bahwa aktivitas transaksi dalam dataset ini berlangsung selama 373 hari, dimulai dari 1 Desember 2010 hingga berakhir pada 9 Desember 2011. Temuan ini mengonfirmasi bahwa dataset ini merupakan rekam jejak historis, yang sekaligus memvalidasi keputusan teknis saya untuk menetapkan tanggal simulasi analisis (snapshot date) pada Desember 2011 dan tidak menggunakan menggunakan tanggal hari ini.

In [ ]:
import datetime as bahlil

# snapshot = Data terakhir + 1 hari = 09 Desember 2011 + 1 Hari = 10 Desember 2011
snapshot_date = df_clean['InvoiceDate'].max() + bahlil.timedelta(days=1)

print(f"Analisis ini disimulasikan pada tanggal: {snapshot_date.date()}")

# Grouping per CustomerID
rfm = df_clean.groupby('CustomerID').agg({
    'InvoiceDate': lambda x: (snapshot_date - x.max()).days, # Recency: Jarak hari belanja terakhir
    'InvoiceNo': 'nunique',  # Frequency: Jumlah struk unik
    'TotalPrice': 'sum'      # Monetary: Total uang yang dibelanjakan
})

rfm.rename(columns={
    'InvoiceDate': 'Recency',
    'InvoiceNo': 'Frequency',
    'TotalPrice': 'Monetary'
}, inplace=True)

print("\nHasil RFM:")
rfm.head()

Analisis ini disimulasikan pada tanggal: 2011-12-10

Hasil RFM:


,Recency,Frequency,Monetary
CustomerID,,,
12346,326,1,77183.60
12347,2,7,4310.00
12348,75,4,1797.24
12349,19,1,1757.55
12350,310,1,334.40
